In [1]:
import pickle
import shutil
import os

import xlsxwriter
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

/home/michel/.virtualenvs/rome_naf/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/michel/.virtualenvs/rome_naf/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


# Charge les référentiels
cf referentiels.py

In [2]:
ROME_df = pd.read_csv('../referentiels/referentiel_ROME/20150921_arboprincipale28427_ROME.csv', index_col=0, sep='|', dtype=str)
OGR_df = pd.read_csv('../referentiels/referentiel_OGR/20150921_arboprincipale28427_OGR.csv', sep='|', dtype=str).set_index('OGR')
NAF_df = pd.read_csv('../referentiels/referentiel_NAF/naf2008_liste_n5_nouveau_header.csv', sep='|', encoding="utf-8").set_index(['NAF'])

# Charge les données

In [3]:
with open('array_ROME1_ETT.pickle', 'rb') as f:
    ROME1_ETT = pickle.load(f)
with open('array_ROME1_fusion.pickle', 'rb') as f:
    ROME1_fusion = pickle.load(f)
with open('array_ROME1_CDI_CDD.pickle', 'rb') as f:
    ROME1_CDI_CDD = pickle.load(f)
with open('array_ROME2_ETT.pickle', 'rb') as f:
    ROME2_ETT = pickle.load(f)
with open('array_ROME2_fusion.pickle', 'rb') as f:
    ROME2_fusion = pickle.load(f)
with open('array_ROME2_CDI_CDD.pickle', 'rb') as f:
    ROME2_CDI_CDD = pickle.load(f)
with open('array_offres.pickle', 'rb') as f:
    offres = pickle.load(f)
with open('array_offres_OGR.pickle', 'rb') as f:
    offres_OGR = pickle.load(f)


# ROME D1106

In [4]:
ROME = 'D1106'

ROME_index = ROME_df.index.get_loc(ROME)
ROME_label = ROME_df.loc[ROME].label

In [26]:
ROME_OGR = OGR_df[OGR_df.ROME == ROME]

In [64]:
OGR_weiths = np.array([offres_OGR[:, OGR_df.index.get_loc(OGR)].sum() for OGR in ROME_OGR.index])
OGR_ordered = [ROME_OGR.index[i] for i in (-OGR_weiths).argsort()]

for OGR in OGR_ordered:
    OGR_index = OGR_df.index.get_loc(OGR)
    OGR_label = OGR_df.iloc[OGR_index].label
    
    print("OGR {} : {}".format(OGR, OGR_label))
    
    nb_offers_by_NAF = offres_OGR[:, OGR_index]
    NAF_indexes_order = (-nb_offers_by_NAF).argsort()

    for NAF_index in NAF_indexes_order[:10]:
        NAF = NAF_df.index[NAF_index]
        NAF_label = NAF_df.iloc[NAF_index].label
        
        nb_offers = offres_OGR[NAF_index, OGR_index]
        
        if nb_offers:
            print("\t{}\t{}\t{}".format(int(nb_offers), NAF, NAF_label))
    
    print()

OGR 20560 : Vendeur / Vendeuse en boulangerie-pâtisserie
	27859	1071C	Boulangerie et boulangerie-pâtisserie
	2226	4724Z	Commerce de détail de pain, pâtisserie et confiserie en magasin spécialisé
	2168	7010Z	Activités des sièges sociaux
	1968	8559A	Formation continue d'adultes
	1926	1071B	Cuisson de produits de boulangerie
	1823	5610C	Restauration de type rapide
	1667	1071D	Pâtisserie
	1191	7820Z	Activités des agences de travail temporaire
	665	9411Z	Activités des organisations patronales et consulaires
	573	4711D	Supermarchés

OGR 20564 : Vendeur / Vendeuse en charcuterie
	1247	4711D	Supermarchés
	934	4711F	Hypermarchés
	642	7820Z	Activités des agences de travail temporaire
	609	1013B	Charcuterie
	352	4722Z	Commerce de détail de viandes et de produits à base de viande en magasin spécialisé
	105	8559A	Formation continue d'adultes
	90	7010Z	Activités des sièges sociaux
	74	4711B	Commerce d'alimentation générale
	71	4781Z	Commerce de détail alimentaire sur éventaires et marchés
	70	8299Z	

In [65]:
NAF_weiths = np.zeros_like(offres_OGR[:, 0])
for OGR in ROME_OGR.index:
    OGR_index = OGR_df.index.get_loc(OGR)
    NAF_weiths += offres_OGR[:, OGR_index]
NAF_indexes_order = (-NAF_weiths).argsort()

OGR_indexes_list = [OGR_df.index.get_loc(OGR) for OGR in ROME_OGR.index]

for NAF_index in NAF_indexes_order:
    NAF = NAF_df.index[NAF_index]
    NAF_label = NAF_df.iloc[NAF_index].label
    
    print("NAF {} : {}".format(NAF, NAF_label))
    
    nb_offers_by_OGR = np.array([offres_OGR[NAF_index, OGR_index] for OGR_index in OGR_indexes_list])
    OGR_indexes_order = [OGR_indexes_list[i] for i in (-nb_offers_by_OGR).argsort()]

    for OGR_index in OGR_indexes_order[:10]:
        OGR = OGR_df.index[OGR_index]
        OGR_label = OGR_df.iloc[OGR_index].label
        
        nb_offers = offres_OGR[NAF_index, OGR_index]
        
        if nb_offers:
            print("\t{}\t{}\t{}".format(int(nb_offers), OGR, OGR_label))
    
    print()

NAF 1071C : Boulangerie et boulangerie-pâtisserie
	27859	20560	Vendeur / Vendeuse en boulangerie-pâtisserie
	83	20540	Vendeur / Vendeuse en alimentation générale
	33	14863	Epicier / Epicière
	33	20567	Vendeur / Vendeuse en chocolaterie
	13	20646	Vendeur / Vendeuse rayon traiteur
	6	20576	Vendeur / Vendeuse en épicerie
	5	20564	Vendeur / Vendeuse en charcuterie
	4	20623	Vendeur / Vendeuse en produits frais
	4	38776	Vendeur / Vendeuse en produits biologiques
	2	20558	Vendeur / Vendeuse en boucherie

NAF 4729Z : Autres commerces de détail alimentaires en magasin spécialisé
	1517	38776	Vendeur / Vendeuse en produits biologiques
	1056	20577	Vendeur / Vendeuse en épicerie fine
	523	20622	Vendeur / Vendeuse en produits diététiques
	468	20576	Vendeur / Vendeuse en épicerie
	298	20540	Vendeur / Vendeuse en alimentation générale
	220	20645	Vendeur / Vendeuse rayon fromages
	148	20570	Vendeur / Vendeuse en crèmerie
	125	20560	Vendeur / Vendeuse en boulangerie-pâtisserie
	120	20525	Vendeur / Vende